# Packages

In [1]:
# Data Processing and Analytics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
# Machine Learning and ML Preprocessing
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
# Neural Networks
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import SparseCategoricalCrossentropy, BinaryCrossentropy
from tensorflow.keras.regularizers import L2, L1
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
# Presets
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
warnings.filterwarnings('ignore')
%matplotlib inline

## Set Seed

In [2]:
np.random.seed(100)

# Import Data

In [3]:
data = pd.read_csv('data_for_ml.csv')
data.head(2)

,INDEX_,HOUR,DAYOFWEEK,VISIBILITY,LIGHT,RDSFCOND,TEMP,REL_HUMID,LOCCOORD,TRAFFCTL,ROADCLASS,SPEEDLMT,VEH_ADT,PED_ADT,LAND_USE,POP_2021,PRIV_DWELL,LAND_AREA,INVAGE,PEDCOND,PEDACT,VEHINV,VIOL,INJURY
0,3366652,7,weekday,Other,Dark,Wet,1.5,0.99,Midblock,No Control,Major Arterial,60,498.0,219.0,Mixed Use,504,263,225,45 to 64,Distracted,Crossing without ROW,automobile,speeding,Major
1,3370334,19,weekday,Rain,Dark,Wet,4.8,1.00,Midblock,Traffic Signal including Transit,Major Arterial,60,351.0,63.0,Residential,452,205,183,Over 65,Normal,Crossing without ROW,automobile,speeding,Major


In [4]:
df = data.drop('INDEX_', axis=1)
df.head(1)

,HOUR,DAYOFWEEK,VISIBILITY,LIGHT,RDSFCOND,TEMP,REL_HUMID,LOCCOORD,TRAFFCTL,ROADCLASS,SPEEDLMT,VEH_ADT,PED_ADT,LAND_USE,POP_2021,PRIV_DWELL,LAND_AREA,INVAGE,PEDCOND,PEDACT,VEHINV,VIOL,INJURY
0,7,weekday,Other,Dark,Wet,1.5,0.99,Midblock,No Control,Major Arterial,60,498.0,219.0,Mixed Use,504,263,225,45 to 64,Distracted,Crossing without ROW,automobile,speeding,Major


In [6]:
selected_features = ['HOUR', 'TEMP', 'REL_HUMID', 'VEH_ADT', 'PED_ADT', 'POP_2021', 'PRIV_DWELL', 'LAND_AREA', 'INVAGE', 'PEDCOND', 'INJURY']
new_df = df[selected_features]
new_df.head(1)

,HOUR,TEMP,REL_HUMID,VEH_ADT,PED_ADT,POP_2021,PRIV_DWELL,LAND_AREA,INVAGE,PEDCOND,INJURY
0,7,1.5,0.99,498.0,219.0,504,263,225,45 to 64,Distracted,Major


In [7]:
# Integer encode the INJURY column
injury_map = {
    'Fatal' : 4,
    'Major' : 3,
    'Minor' : 2,
    'Minimal' : 1,
    np.nan : 0
}
new_df['INJURY'] = new_df['INJURY'].replace(injury_map).astype(int)
new_df.head(1)

,HOUR,TEMP,REL_HUMID,VEH_ADT,PED_ADT,POP_2021,PRIV_DWELL,LAND_AREA,INVAGE,PEDCOND,INJURY
0,7,1.5,0.99,498.0,219.0,504,263,225,45 to 64,Distracted,3


# ML Data Preprocessing

In [8]:
# Split into X and y variables
X = new_df.iloc[:, :-1].values
y = new_df.iloc[:,-1].values

In [9]:
# One hot encode the categorical columns
cat_columns_index = [8, 9]
encoder = OneHotEncoder(categories='auto', sparse=False, handle_unknown='ignore')
X_cat = encoder.fit_transform(X[:, cat_columns_index])

In [10]:
# Combine the categorical and numerical columns
num_columns_index = [0, 1, 2, 3, 4, 5, 6, 7]
X_encoded = np.concatenate((X_cat, X[:, num_columns_index].astype(float)), axis=1)
print(X_encoded)

[[0.000e+00 0.000e+00 1.000e+00 ... 5.040e+02 2.630e+02 2.250e+02]
 [0.000e+00 0.000e+00 0.000e+00 ... 4.520e+02 2.050e+02 1.830e+02]
 [1.000e+00 0.000e+00 0.000e+00 ... 8.070e+02 3.750e+02 3.220e+02]
 ...
 [0.000e+00 0.000e+00 1.000e+00 ... 4.820e+02 2.520e+02 2.310e+02]
 [0.000e+00 0.000e+00 0.000e+00 ... 1.981e+03 1.195e+03 1.112e+03]
 [0.000e+00 0.000e+00 1.000e+00 ... 1.170e+03 4.820e+02 4.670e+02]]


In [11]:
# Split encoded X and y to train test split
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.20, random_state=100)

In [12]:
# Scale the numerical features
scaler = StandardScaler()
X_train[:, -len(num_columns_index):] = scaler.fit_transform(X_train[:, -len(num_columns_index):])
X_test[:, -len(num_columns_index):] = scaler.transform(X_test[:, -len(num_columns_index):])
print(f"X_train dimension: {X_train.shape}")
print(f"y_train dimension: {y_train.shape}")
print(f"X_test dimension: {X_test.shape}")
print(f"y_test dimension: {y_test.shape}")

X_train dimension: (2490, 23)
y_train dimension: (2490,)
X_test dimension: (623, 23)
y_test dimension: (623,)


# ML Modeling and Metrics

In [13]:
models = [KNeighborsClassifier(), RandomForestClassifier(), XGBClassifier()]

In [14]:
# Define parameter grids
param_grid_knn = {
    'n_neighbors' : [i for i in range(1,36)],
    'weights' : ['uniform', 'distance'],
    'metric' : ['euclidean', 'manhattan']
}

param_grid_rf = {
    'n_estimators': [50, 100, 200],                
    'max_features': ['auto', 'sqrt', 'log2'],      
    'max_depth': [None, 10, 20, 30],               
    'min_samples_split': [2, 5, 10],               
    'min_samples_leaf': [1, 2, 4],                 
    'bootstrap': [True, False],                    
    'criterion': ['gini', 'entropy']               
}

param_grid_xgb = {
    'learning_rate': [0.01, 0.1, 0.3],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.5, 0.8, 1.0],
    'colsample_bytree': [0.5, 0.8, 1.0]
}

param_grids = [param_grid_knn, param_grid_rf, param_grid_xgb]

In [15]:
# Iterate on both grids and models to fit the model with our training data
for model, grid in zip(models, param_grids):
    random_search = RandomizedSearchCV(estimator=model,
                                      param_distributions=grid,
                                      n_iter=100,
                                      cv=5,
                                      scoring='accuracy',
                                      random_state=100)
    random_search.fit(X_train, y_train)
    y_pred = random_search.predict(X_test)
    training_accuracy = random_search.best_score_
    best_parameters = random_search.best_params_
    print(f'{model} Modeling Results:\n')
    print('Best Parameters:')
    print(f'{best_parameters}\n')
    print(f"Training Accuracy: {training_accuracy*100:.2f}%")
    print(f"Testing Accuracy: {accuracy_score(y_test, y_pred)*100:.2f}%")
    print(f"Precision: {precision_score(y_test, y_pred, average='weighted')*100:.2f}%")
    print(f"Recall: {recall_score(y_test, y_pred, average='weighted')*100:.2f}%")
    print(f"F1 Score: {f1_score(y_test, y_pred, average='weighted')*100:.2f}%\n")
    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    print(f"Confusion Matrix:")
    print(f"{cm}\n")

KNeighborsClassifier() Modeling Results:

Best Parameters:
{'weights': 'uniform', 'n_neighbors': 17, 'metric': 'euclidean'}

Training Accuracy: 77.75%
Testing Accuracy: 78.65%
Precision: 69.67%
Recall: 78.65%
F1 Score: 70.70%

Confusion Matrix:
[[  0   0   0   6   0]
 [  0   0   0  14   0]
 [  0   0   0  18   2]
 [  0   0   0 484   4]
 [  0   0   0  89   6]]

RandomForestClassifier() Modeling Results:

Best Parameters:
{'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': 10, 'criterion': 'gini', 'bootstrap': True}

Training Accuracy: 77.99%
Testing Accuracy: 78.49%
Precision: 69.06%
Recall: 78.49%
F1 Score: 70.61%

Confusion Matrix:
[[  0   0   0   6   0]
 [  0   0   0  14   0]
 [  0   0   0  18   2]
 [  0   0   0 483   5]
 [  0   0   0  89   6]]

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_ro

# Neural Network Modeling

In [16]:
tf.random.set_seed(100)

## Design 1 - 2 Hidden Layers

In [30]:
# Construct NN Architecture
design1 = Sequential([
    Dense(units=10, activation='relu', input_shape=(23,)),
    Dropout(rate=0.1),
    Dense(units=5, activation='linear', kernel_regularizer=L1(0.01))
])
# Compile the model
design1.compile(loss=SparseCategoricalCrossentropy(from_logits=True),
               optimizer=Adam(learning_rate=0.01),
               metrics=['accuracy'])
# Train the model
design1.fit(X_train, y_train, epochs=100, validation_data=[X_test, y_test])

Epoch 1/100
78/78 [==============================] - 1s 3ms/step - loss: 1.0057 - accuracy: 0.7265 - val_loss: 0.8003 - val_accuracy: 0.7833
Epoch 2/100
78/78 [==============================] - 0s 2ms/step - loss: 0.7743 - accuracy: 0.7751 - val_loss: 0.7380 - val_accuracy: 0.7833
Epoch 3/100
78/78 [==============================] - 0s 2ms/step - loss: 0.7404 - accuracy: 0.7735 - val_loss: 0.7225 - val_accuracy: 0.7833
Epoch 4/100
78/78 [==============================] - 0s 2ms/step - loss: 0.7122 - accuracy: 0.7763 - val_loss: 0.7068 - val_accuracy: 0.7817
Epoch 5/100
78/78 [==============================] - 0s 2ms/step - loss: 0.7084 - accuracy: 0.7799 - val_loss: 0.7025 - val_accuracy: 0.7833
Epoch 6/100
78/78 [==============================] - 0s 2ms/step - loss: 0.6997 - accuracy: 0.7799 - val_loss: 0.7265 - val_accuracy: 0.7833
Epoch 7/100
78/78 [==============================] - 0s 2ms/step - loss: 0.7100 - accuracy: 0.7787 - val_loss: 0.6985 - val_accuracy: 0.7833
Epoch 8/100
7

Epoch 59/100
78/78 [==============================] - 0s 2ms/step - loss: 0.6636 - accuracy: 0.7775 - val_loss: 0.6883 - val_accuracy: 0.7817
Epoch 60/100
78/78 [==============================] - 0s 2ms/step - loss: 0.6651 - accuracy: 0.7747 - val_loss: 0.6950 - val_accuracy: 0.7769
Epoch 61/100
78/78 [==============================] - 0s 2ms/step - loss: 0.6646 - accuracy: 0.7759 - val_loss: 0.6923 - val_accuracy: 0.7753
Epoch 62/100
78/78 [==============================] - 0s 2ms/step - loss: 0.6674 - accuracy: 0.7755 - val_loss: 0.6894 - val_accuracy: 0.7817
Epoch 63/100
78/78 [==============================] - 0s 2ms/step - loss: 0.6609 - accuracy: 0.7767 - val_loss: 0.6905 - val_accuracy: 0.7881
Epoch 64/100
78/78 [==============================] - 0s 2ms/step - loss: 0.6637 - accuracy: 0.7739 - val_loss: 0.6893 - val_accuracy: 0.7849
Epoch 65/100
78/78 [==============================] - 0s 2ms/step - loss: 0.6647 - accuracy: 0.7751 - val_loss: 0.6961 - val_accuracy: 0.7817
Epoch 

## Design 2 - 4 Hidden Layers

In [32]:
# Construct NN Architecture
design2 = Sequential([
    Dense(units=10, activation='relu', input_shape=(23,)),
    Dropout(rate=0.1),
    Dense(units=10, activation='relu'),
    Dropout(rate=0.1),
    Dense(units=5, activation='linear', kernel_regularizer=L1(0.01))
])
# Compile the model
design2.compile(loss=SparseCategoricalCrossentropy(from_logits=True),
               optimizer=Adam(learning_rate=0.01),
               metrics=['accuracy'])
# Train the model
design2.fit(X_train, y_train, epochs=100, validation_data=[X_test, y_test])

Epoch 1/100
78/78 [==============================] - 1s 3ms/step - loss: 1.0142 - accuracy: 0.7313 - val_loss: 0.7929 - val_accuracy: 0.7833
Epoch 2/100
78/78 [==============================] - 0s 2ms/step - loss: 0.7768 - accuracy: 0.7751 - val_loss: 0.7439 - val_accuracy: 0.7833
Epoch 3/100
78/78 [==============================] - 0s 2ms/step - loss: 0.7498 - accuracy: 0.7751 - val_loss: 0.7449 - val_accuracy: 0.7833
Epoch 4/100
78/78 [==============================] - 0s 2ms/step - loss: 0.7353 - accuracy: 0.7751 - val_loss: 0.7146 - val_accuracy: 0.7833
Epoch 5/100
78/78 [==============================] - 0s 2ms/step - loss: 0.7220 - accuracy: 0.7751 - val_loss: 0.7030 - val_accuracy: 0.7833
Epoch 6/100
78/78 [==============================] - 0s 2ms/step - loss: 0.7033 - accuracy: 0.7755 - val_loss: 0.7279 - val_accuracy: 0.7833
Epoch 7/100
78/78 [==============================] - 0s 2ms/step - loss: 0.7066 - accuracy: 0.7771 - val_loss: 0.6979 - val_accuracy: 0.7833
Epoch 8/100
7

Epoch 59/100
78/78 [==============================] - 0s 2ms/step - loss: 0.6563 - accuracy: 0.7767 - val_loss: 0.6854 - val_accuracy: 0.7881
Epoch 60/100
78/78 [==============================] - 0s 2ms/step - loss: 0.6540 - accuracy: 0.7791 - val_loss: 0.6895 - val_accuracy: 0.7801
Epoch 61/100
78/78 [==============================] - 0s 2ms/step - loss: 0.6593 - accuracy: 0.7739 - val_loss: 0.6879 - val_accuracy: 0.7817
Epoch 62/100
78/78 [==============================] - 0s 2ms/step - loss: 0.6626 - accuracy: 0.7751 - val_loss: 0.6879 - val_accuracy: 0.7865
Epoch 63/100
78/78 [==============================] - 0s 2ms/step - loss: 0.6521 - accuracy: 0.7799 - val_loss: 0.6825 - val_accuracy: 0.7833
Epoch 64/100
78/78 [==============================] - 0s 2ms/step - loss: 0.6494 - accuracy: 0.7775 - val_loss: 0.6792 - val_accuracy: 0.7817
Epoch 65/100
78/78 [==============================] - 0s 2ms/step - loss: 0.6531 - accuracy: 0.7763 - val_loss: 0.6974 - val_accuracy: 0.7801
Epoch 

## Design 3 - 6 Hidden Layers

In [33]:
# Construct NN Architecture
design3 = Sequential([
    Dense(units=10, activation='relu', input_shape=(23,)),
    Dropout(rate=0.1),
    Dense(units=10, activation='relu'),
    Dropout(rate=0.1),
    Dense(units=10, activation='relu'),
    Dropout(rate=0.1),
    Dense(units=5, activation='linear', kernel_regularizer=L1(0.01))
])
# Compile the model
design3.compile(loss=SparseCategoricalCrossentropy(from_logits=True),
               optimizer=Adam(learning_rate=0.01),
               metrics=['accuracy'])
# Train the model
design3.fit(X_train, y_train, epochs=100, validation_data=[X_test, y_test])

Epoch 1/100
78/78 [==============================] - 1s 4ms/step - loss: 0.9645 - accuracy: 0.7438 - val_loss: 0.7928 - val_accuracy: 0.7833
Epoch 2/100
78/78 [==============================] - 0s 2ms/step - loss: 0.7738 - accuracy: 0.7751 - val_loss: 0.7467 - val_accuracy: 0.7833
Epoch 3/100
78/78 [==============================] - 0s 2ms/step - loss: 0.7421 - accuracy: 0.7751 - val_loss: 0.7329 - val_accuracy: 0.7833
Epoch 4/100
78/78 [==============================] - 0s 2ms/step - loss: 0.7282 - accuracy: 0.7751 - val_loss: 0.7143 - val_accuracy: 0.7833
Epoch 5/100
78/78 [==============================] - 0s 2ms/step - loss: 0.7187 - accuracy: 0.7727 - val_loss: 0.7036 - val_accuracy: 0.7865
Epoch 6/100
78/78 [==============================] - 0s 2ms/step - loss: 0.7002 - accuracy: 0.7727 - val_loss: 0.7144 - val_accuracy: 0.7833
Epoch 7/100
78/78 [==============================] - 0s 2ms/step - loss: 0.6979 - accuracy: 0.7755 - val_loss: 0.6896 - val_accuracy: 0.7833
Epoch 8/100
7

Epoch 59/100
78/78 [==============================] - 0s 2ms/step - loss: 0.6538 - accuracy: 0.7799 - val_loss: 0.7199 - val_accuracy: 0.7801
Epoch 60/100
78/78 [==============================] - 0s 2ms/step - loss: 0.6490 - accuracy: 0.7775 - val_loss: 0.7176 - val_accuracy: 0.7785
Epoch 61/100
78/78 [==============================] - 0s 2ms/step - loss: 0.6498 - accuracy: 0.7771 - val_loss: 0.7364 - val_accuracy: 0.7785
Epoch 62/100
78/78 [==============================] - 0s 2ms/step - loss: 0.6614 - accuracy: 0.7767 - val_loss: 0.7198 - val_accuracy: 0.7737
Epoch 63/100
78/78 [==============================] - 0s 2ms/step - loss: 0.6435 - accuracy: 0.7775 - val_loss: 0.7125 - val_accuracy: 0.7753
Epoch 64/100
78/78 [==============================] - 0s 2ms/step - loss: 0.6518 - accuracy: 0.7763 - val_loss: 0.7135 - val_accuracy: 0.7801
Epoch 65/100
78/78 [==============================] - 0s 2ms/step - loss: 0.6496 - accuracy: 0.7803 - val_loss: 0.7367 - val_accuracy: 0.7769
Epoch 

# NN Performance Metrics

In [34]:
models = [design1, design2, design3]
for model in models:
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    train_loss, train_accuracy = model.evaluate(X_train, y_train)
    test_loss, test_accuracy = model.evaluate(X_test, y_test)
    precision = precision_score(y_test, y_pred_classes, average='weighted')
    recall = recall_score(y_test, y_pred_classes, average='weighted')
    f1 = f1_score(y_test, y_pred_classes, average='weighted')
    print(f"Model Performance Metrics:")
    print(f"Training Accuracy: {train_accuracy*100:.2f}%")
    print(f"Test Accuracy: {test_accuracy*100:.2f}%")
    print(f"Precision: {precision*100:.2f}%")
    print(f"Recall: {recall*100:.2f}%")
    print(f"F1 Score: {f1*100:.2f}%\n")
    print(f"Confusion Matrix:")
    print(f"{confusion_matrix(y_test, y_pred_classes)}\n")

20/20 [==============================] - 0s 1ms/step - loss: 0.6929 - accuracy: 0.7769
Model Performance Metrics:
Training Accuracy: 78.39%
Test Accuracy: 77.69%
Precision: 68.16%
Recall: 77.69%
F1 Score: 71.19%

Confusion Matrix:
[[  0   0   0   6   0]
 [  0   0   0  14   0]
 [  0   0   0  17   3]
 [  0   0   0 473  15]
 [  0   0   0  84  11]]

20/20 [==============================] - 0s 969us/step - loss: 0.6856 - accuracy: 0.7801
Model Performance Metrics:
Training Accuracy: 78.23%
Test Accuracy: 78.01%
Precision: 67.95%
Recall: 78.01%
F1 Score: 70.55%

Confusion Matrix:
[[  0   0   0   6   0]
 [  0   0   0  14   0]
 [  0   0   0  18   2]
 [  0   0   0 479   9]
 [  0   0   0  88   7]]

20/20 [==============================] - 0s 1ms/step - loss: 0.7531 - accuracy: 0.7769
Model Performance Metrics:
Training Accuracy: 77.87%
Test Accuracy: 77.69%
Precision: 63.60%
Recall: 77.69%
F1 Score: 68.84%

Confusion Matrix:
[[  0   0   0   6   0]
 [  0   0   0  14   0]
 [  0   0   0  19   1]
 [